In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

import missingno as msno
import pandas_profiling

In [2]:
filepath = './Accepted 2007 to 2018 Q4_New.csv.gz'
data_raw = pd.read_csv(filepath, compression = 'gzip')

D:\Anaconda\Anaconda_3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option("display.max_columns", 150)

In [4]:
data_raw.shape

(2260701, 151)

##### Define a get_na function

In [5]:
## this function will return two dataframes of columns and rows over the defined threthold (in % scale)
def get_na(df, threshold = 95):
    
    # getting the couunt and percentage of missingness for rows
    count_row = df.isnull().sum(axis = 1).sort_values(ascending = False)
    perc_row = np.round(count_row / df.shape[1] * 100, 2)
    # getting the df of largely missing rows
    df_row = pd.DataFrame({'Count': count_row, 'Percent': perc_row})
    df_row = df_row[df_row.Percent >= threshold]
    
    # getting the couunt and percentage of missingness for columns
    count_col = df.isnull().sum().sort_values(ascending = False)
    perc_col = np.round(count_col / df.shape[0] * 100, 2)
    # getting the df of largely missing columns
    df_col = pd.DataFrame({'Count':count_col, 'Percent': perc_col})
    df_col = df_col[df_col.Percent >= threshold]
    
    return df_row, df_col

In [6]:
## list of selected features, considering missingness, domain knowledge and corr test
lst_feature = ['acc_now_delinq',
'addr_state',
'all_util',
'annual_inc',
'application_type',
'chargeoff_within_12_mths',
'collections_12_mths_ex_med',
'delinq_2yrs',
'delinq_amnt',
'dti',
'earliest_cr_line',
'emp_length',
'fico_range_high',
'fico_range_low',
'funded_amnt',
'grade',
'home_ownership',
'il_util',
'initial_list_status',
'inq_last_12m',
'int_rate',
'issue_d',
'mo_sin_rcnt_tl',
'mths_since_last_delinq',
'mths_since_last_major_derog',
'mths_since_last_record',
'num_accts_ever_120_pd',
'num_actv_rev_tl',
'num_rev_accts',
'num_tl_90g_dpd_24m',
'num_tl_op_past_12m',
'open_acc',
'open_act_il',
'num_il_tl',
'pct_tl_nvr_dlq',
'pub_rec',
'pub_rec_bankruptcies',
'purpose',
'revol_util',
'sub_grade',
'tax_liens',
'term',
'tot_coll_amt',
'tot_cur_bal',
'tot_hi_cred_lim',
'total_acc',
'total_bc_limit',
'total_il_high_credit_limit',
'total_rev_hi_lim',
'verification_status',
'collection_recovery_fee',
'last_pymnt_d',
'loan_status',
'settlement_status',
'total_pymnt']

In [7]:
len(lst_feature)

55

In [8]:
## only to keep selected features
data = data_raw[lst_feature]

## drop rows with large missingness
df_na_r, df_na_c = get_na(data)
lst_na_r = df_na_r.index.tolist()
data = data.drop(lst_na_r, axis = 0)
data.shape

(2260668, 55)

### Feature Engineering and Processing

In [9]:
## create the fico score col and drop high/low range
data['fico_score'] = (data.fico_range_high + data.fico_range_low) / 2
data.drop(['fico_range_high', 'fico_range_low'], axis = 1, inplace = True)

## create fractions of active revolving and installment accounts
data['active_rev_acct_frac'] = data.num_actv_rev_tl / data.num_rev_accts
data['active_ins_acct_frac'] = data.open_act_il /  data.num_il_tl
data.drop(['num_actv_rev_tl', 'open_act_il'], axis = 1, inplace = True)

## calculate gains
data['total_payout'] = np.round(data.total_pymnt - data.collection_recovery_fee, 2)
data.drop(['total_pymnt', 'collection_recovery_fee'], axis = 1, inplace = True)

## exclude loans with ACTIVE settlement status and then drop the settlement status col
data[data.settlement_status != 'ACTIVE']
data.drop('settlement_status', axis = 1, inplace = True)

## consolidate loan statuses into: Current, Default & Fully Paid
lst_curr = ['Current', 'In Grace Period', 'Late (31-120 days)', 'Late (16-30 days)']
lst_def = ['Charged Off', 'Default', 'Does not meet the credit policy. Status:Charged Off']
lst_fp = ['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']
data.loan_status = ['Current' if x in lst_curr else 'Default' if x in lst_def else 'Fully Paid' for x in data.loan_status]

## remove rows with missing issue date
data = data[~ data.last_pymnt_d.isnull()]

## converying dates to datetime objects
data.issue_d = pd.to_datetime(data.issue_d)
data.last_pymnt_d = pd.to_datetime(data.last_pymnt_d)
data.earliest_cr_line = pd.to_datetime(data.earliest_cr_line)

## calculate duration -> convert to int & drop last payment date
data['duration'] = (data.last_pymnt_d - data.issue_d).dt.days / 365 + 1 / 12   # plus one mon to ensure there is no 0 duration
data.drop('last_pymnt_d', axis = 1, inplace = True)

## calculate annualized return rate
data['return_rate'] = (data.total_payout / data.funded_amnt) ** (1 / data.duration) - 1

## convert earliest_cr_line to credit history (in years)
data['credit_hist'] = (data.issue_d - data.earliest_cr_line).dt.days / 365
data.drop('earliest_cr_line', axis = 1, inplace = True)

## convert term to numeric numbers
data.term = data.term.str.extract('(\d+)').astype('int64')

## convert employment length into numeric
data.emp_length = data.emp_length.str.extract('(\d+)').fillna(value = -999, axis = 1).astype('int64')

#### Add State-level Monthly Unemployment Rate (3-Mon MV) and Monthly 3-Yr & 5-Yr Treasury Rate to Dataset

In [10]:
treasury = pd.read_csv('Monthly Treasury Rate Since 2007 (3- & 5-Yr Combined).csv')
unemp = pd.read_csv('Unemployment Rate_State.csv')

In [11]:
## convert date features to datetime format
treasury.month = pd.to_datetime(treasury.month)
unemp.mon_year = pd.to_datetime(unemp.mon_year)

## create a unique identifier
treasury['id'] = treasury.month.map(str) + treasury.term.map(str)
unemp['id'] = unemp.mon_year.map(str) + unemp.state

## drop irrelavant columns
treasury.drop(['month', 'term'], axis = 1, inplace = True)
unemp.drop(['state', 'mon_year', 'unemp_rate'], axis = 1, inplace = True)

In [12]:
## generate the same identifier in main dataframe to match treasury rates
data['id'] = data.issue_d.map(str) + data.term.map(str)
data = data.merge(treasury, how = 'left', on = 'id')

## update the identifier to match unemployment rates
data['id'] = data.issue_d.map(str) + data.addr_state
data = data.merge(unemp, how = 'left', on = 'id').drop(['id'], axis = 1)

In [13]:
data.head()

,acc_now_delinq,addr_state,all_util,annual_inc,application_type,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,emp_length,funded_amnt,grade,home_ownership,il_util,initial_list_status,inq_last_12m,int_rate,issue_d,mo_sin_rcnt_tl,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,num_accts_ever_120_pd,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,num_il_tl,pct_tl_nvr_dlq,pub_rec,pub_rec_bankruptcies,purpose,revol_util,sub_grade,tax_liens,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bc_limit,total_il_high_credit_limit,total_rev_hi_lim,verification_status,loan_status,fico_score,active_rev_acct_frac,active_ins_acct_frac,total_payout,duration,return_rate,credit_hist,treasury_rate,unemp_rate_3mon
0,0.0,PA,34.0,55000.0,Individual,0.0,0.0,0.0,0.0,5.91,10,3600.0,C,MORTGAGE,36.0,w,4.0,13.99,2015-12-01,3.0,30.0,30.0,NaN,2.0,9.0,0.0,3.0,7.0,3.0,76.9,0.0,0.0,debt_consolidation,29.7,C4,0.0,36,722.0,144904.0,178050.0,13.0,2400.0,13734.0,9300.0,Not Verified,Fully Paid,677.0,0.444444,0.666667,4421.72,3.171005,0.066984,12.342466,1.28,4.77
1,0.0,SD,29.0,65000.0,Individual,0.0,0.0,1.0,0.0,16.06,10,24700.0,C,MORTGAGE,73.0,w,6.0,11.99,2015-12-01,2.0,6.0,NaN,NaN,0.0,27.0,0.0,2.0,22.0,6.0,97.4,0.0,0.0,small_business,19.2,C1,0.0,36,0.0,204396.0,314017.0,38.0,79300.0,24667.0,111800.0,Not Verified,Fully Paid,717.0,0.185185,0.166667,25679.66,0.584703,0.068785,16.010959,1.28,2.73
2,0.0,IL,65.0,63000.0,Joint App,0.0,0.0,0.0,0.0,10.78,10,20000.0,B,MORTGAGE,73.0,w,1.0,10.78,2015-12-01,14.0,NaN,NaN,NaN,0.0,7.0,0.0,0.0,6.0,6.0,100.0,0.0,0.0,home_improvement,56.2,B4,0.0,60,0.0,189699.0,218418.0,18.0,6200.0,14877.0,14000.0,Not Verified,Fully Paid,697.0,0.428571,0.166667,22705.92,1.584703,0.083367,15.342466,1.70,5.83
3,0.0,NJ,45.0,110000.0,Individual,0.0,0.0,0.0,0.0,17.06,10,35000.0,C,MORTGAGE,70.0,w,0.0,14.85,2015-12-01,2.0,NaN,NaN,NaN,0.0,13.0,0.0,1.0,13.0,2.0,100.0,0.0,0.0,debt_consolidation,11.6,C5,0.0,60,0.0,301500.0,381215.0,17.0,62500.0,18000.0,67300.0,Source Verified,Current,787.0,0.384615,0.500000,31464.01,3.255936,-0.032181,7.252055,1.70,4.73
4,0.0,PA,78.0,104433.0,Individual,0.0,0.0,1.0,0.0,25.37,3,10400.0,F,MORTGAGE,84.0,w,3.0,22.45,2015-12-01,4.0,12.0,NaN,NaN,0.0,19.0,0.0,4.0,12.0,10.0,96.6,0.0,0.0,major_purchase,64.5,F1,0.0,60,0.0,331730.0,439570.0,35.0,20300.0,88097.0,34000.0,Source Verified,Fully Paid,697.0,0.315789,0.300000,11740.50,0.666895,0.199369,17.512329,1.70,4.77


### Impute Missingness

In [14]:
## get the missingness dfs
df_na_r, df_na_c = get_na(data, threshold = 0)

data = data.fillna(value = -999, axis = 1)

In [15]:
data.head()

,acc_now_delinq,addr_state,all_util,annual_inc,application_type,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,emp_length,funded_amnt,grade,home_ownership,il_util,initial_list_status,inq_last_12m,int_rate,issue_d,mo_sin_rcnt_tl,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,num_accts_ever_120_pd,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,num_il_tl,pct_tl_nvr_dlq,pub_rec,pub_rec_bankruptcies,purpose,revol_util,sub_grade,tax_liens,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bc_limit,total_il_high_credit_limit,total_rev_hi_lim,verification_status,loan_status,fico_score,active_rev_acct_frac,active_ins_acct_frac,total_payout,duration,return_rate,credit_hist,treasury_rate,unemp_rate_3mon
0,0.0,PA,34.0,55000.0,Individual,0.0,0.0,0.0,0.0,5.91,10,3600.0,C,MORTGAGE,36.0,w,4.0,13.99,2015-12-01,3.0,30.0,30.0,-999.0,2.0,9.0,0.0,3.0,7.0,3.0,76.9,0.0,0.0,debt_consolidation,29.7,C4,0.0,36,722.0,144904.0,178050.0,13.0,2400.0,13734.0,9300.0,Not Verified,Fully Paid,677.0,0.444444,0.666667,4421.72,3.171005,0.066984,12.342466,1.28,4.77
1,0.0,SD,29.0,65000.0,Individual,0.0,0.0,1.0,0.0,16.06,10,24700.0,C,MORTGAGE,73.0,w,6.0,11.99,2015-12-01,2.0,6.0,-999.0,-999.0,0.0,27.0,0.0,2.0,22.0,6.0,97.4,0.0,0.0,small_business,19.2,C1,0.0,36,0.0,204396.0,314017.0,38.0,79300.0,24667.0,111800.0,Not Verified,Fully Paid,717.0,0.185185,0.166667,25679.66,0.584703,0.068785,16.010959,1.28,2.73
2,0.0,IL,65.0,63000.0,Joint App,0.0,0.0,0.0,0.0,10.78,10,20000.0,B,MORTGAGE,73.0,w,1.0,10.78,2015-12-01,14.0,-999.0,-999.0,-999.0,0.0,7.0,0.0,0.0,6.0,6.0,100.0,0.0,0.0,home_improvement,56.2,B4,0.0,60,0.0,189699.0,218418.0,18.0,6200.0,14877.0,14000.0,Not Verified,Fully Paid,697.0,0.428571,0.166667,22705.92,1.584703,0.083367,15.342466,1.70,5.83
3,0.0,NJ,45.0,110000.0,Individual,0.0,0.0,0.0,0.0,17.06,10,35000.0,C,MORTGAGE,70.0,w,0.0,14.85,2015-12-01,2.0,-999.0,-999.0,-999.0,0.0,13.0,0.0,1.0,13.0,2.0,100.0,0.0,0.0,debt_consolidation,11.6,C5,0.0,60,0.0,301500.0,381215.0,17.0,62500.0,18000.0,67300.0,Source Verified,Current,787.0,0.384615,0.500000,31464.01,3.255936,-0.032181,7.252055,1.70,4.73
4,0.0,PA,78.0,104433.0,Individual,0.0,0.0,1.0,0.0,25.37,3,10400.0,F,MORTGAGE,84.0,w,3.0,22.45,2015-12-01,4.0,12.0,-999.0,-999.0,0.0,19.0,0.0,4.0,12.0,10.0,96.6,0.0,0.0,major_purchase,64.5,F1,0.0,60,0.0,331730.0,439570.0,35.0,20300.0,88097.0,34000.0,Source Verified,Fully Paid,697.0,0.315789,0.300000,11740.50,0.666895,0.199369,17.512329,1.70,4.77


### Check Outliers

##### Define a function that excludes outliers outside certain stds above the mean

In [16]:
def outlier_remover(df, num_std = 5, cols = None):
   
    # initate an index list for all outliers
    lst_outlier = []
    
    # if no column is given -> remove outliers on all numeric columns
    if cols == None:
        
        # if no column is given then apply removal to all numeric columns
        cols = df.select_dtypes(include = ['int64', 'float64']).columns.tolist()
        
    # remove outliers above defined # of stds
    for col in cols:

        # exclude imputed -999 from calculation
        lst_no_999 = [x for x in df[col] if x != -999]

        # calculate the mean and std of each column
        avg = np.mean(lst_no_999)
        std = np.std(lst_no_999)

        # index list of obs over mean + x * std, aka outlier indexes
        lst_over = df[df[col] > avg + num_std * std].index.tolist()

        lst_outlier.extend(lst_over)
     
    # remove duplicates in the outlier list & get the retained index list
    lst_retain = list(set(df.index) - set(lst_outlier))
    
    return df.loc[lst_retain]
    ## use loc[] and not iloc[] is bc some rows (along with their indexes) have been removed from feature engineering
    

##### Define a clipper function that bounds the outliers

In [17]:
def clipper(df, num_std = 5, cols = None):
    if cols == None:
        # if no column was given then apply to all numeric columns
        cols = df.select_dtypes(include = ['int64', 'float64']).columns.tolist()
    
    for col in cols:
        # exclude imputed -999 from calculation
        lst_no_999 = [x for x in df[col] if x != -999]

        # calculate the mean and std of each column
        avg = np.mean(lst_no_999)
        std = np.std(lst_no_999)
        
        # impute outliers above defined num of stds while keep the lower boundary at -999
        df[col] = np.clip(df[col], -999, avg + num_std * std)
    
    # return imputed dataframe
    return df
        

### Save the entire completed list

In [18]:
%%time
data_clip = clipper(data)

Wall time: 31.3 s


In [19]:
data_completed_clip = data_clip[data_clip.loan_status != 'Current']
data_curr_clip = data_clip[data_clip.loan_status == 'Current']

In [20]:
data_completed_clip.to_csv(path_or_buf = 'Completed Loans_Outliers Clipped.csv.gz',index = False, compression = 'gzip')

In [21]:
data_curr_clip.to_csv(path_or_buf = 'Current Loans_Outliers Clipped.csv.gz',index = False, compression = 'gzip')

In [ ]:
%%time
data_removed = outlier_remover(data)

In [ ]:
data_completed_removed = data_removed[data_removed.loan_status != 'Current']
data_curr_removed = data_removed[data_removed.loan_status == 'Current']

In [ ]:
data_completed_removed.to_csv(path_or_buf = 'Completed Loans_Outliers Removed.csv.gz',index = False, compression = 'gzip')

In [ ]:
data_curr_removed.to_csv(path_or_buf = 'Current Loans_Outliers Removed.csv.gz',index = False, compression = 'gzip')